In [1]:
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1 # 낮을수록 창의력이 낮은 결과
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Korea", country_b="Japan")

chat.predict(prompt)

"The distance between Korea and Japan varies depending on the specific locations being considered. The shortest distance between the two countries is approximately 200 kilometers (124 miles) across the Korea Strait, which separates the southwestern tip of Japan's Kyushu Island from the southeastern tip of South Korea's Busan. However, the distance can be much greater when considering other parts of the countries. For example, the distance between Seoul, the capital of South Korea, and Tokyo, the capital of Japan, is approximately 1,200 kilometers (746 miles) by air."

In [6]:
template = ChatPromptTemplate.from_messages
([
    ("system", "You are a Seller Expert. And you only reply in {language}."),
    ("ai", "안녕, 반가워 {name}!"),
    ("human", "What is the favorite food in {country}?. Also, What's your name?"),
])

prompt = template.format_messages(language = "Japanese", name = "Sangmoo", country = "Korea")

chat.predict_messages(prompt)

AttributeError: 'function' object has no attribute 'format_messages'

In [11]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        # 쉼표 단위로 list 생성
        items = text.strip().split(",") 
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you ?")

['Hello', 'how', 'are', 'you ?']

In [13]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

### predict prompt with CommaOutputParser
# prompt = template.format_messages(max_items = 10, question = "What are the colors?")
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

### Create chain
# use "|" operater 
chain_one = template | chat | CommaOutputParser()

### chain | chain
# chain_tow = template_2 | chat | outputParser
# all = chain_one | chain_tow | output

chain.invoke({
    "max_items" : 5,
    "question" : "What are the digimons?"
})

['agumon', 'gabumon', 'patamon', 'gatomon', 'veemon']